In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks, layers, Model
from SpeechModels import get_melspec_model

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [5]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [6]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels


train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## Model stacked bidirectional lstm

In [7]:
def create_bidirectional_model(number_of_layers):
    mel_spec_model = get_melspec_model(16000)
    inputs, outputs = mel_spec_model.inputs, mel_spec_model.outputs
    x = outputs[0]
    for i in range(0, number_of_layers):
        if i != number_of_layers - 1:
            x = layers.Bidirectional(layers.LSTM(2 ** (8 - i), return_sequences=True))(x)
        else:
            x = layers.Bidirectional(layers.LSTM(2 ** (8 - i)))(x)
    out = layers.Dense(N_CLASS, activation="softmax")(x)
    return Model(inputs=inputs, outputs=out)


model = create_bidirectional_model(1)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft (TFOpLambda)    (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs (TFOpLambda)       (None, 125, 513)     0           ['tf.signal.stft[0][0]']         
                                                                                                  
 tf.tensordot (TFOpLambda)      (None, 125, 80)      0           ['tf.math.abs[0][0]']            
                                                                                              

In [8]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3,
                                        min_lr=0.00001, verbose=1)

In [9]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/2
90/90 [==============================] - 22s 119ms/step - loss: 1.3683 - sparse_categorical_accuracy: 0.6332 - sparse_categorical_crossentropy: 1.3683 - val_loss: 1.0421 - val_sparse_categorical_accuracy: 0.6846 - val_sparse_categorical_crossentropy: 1.0421 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 13s 132ms/step - loss: 0.8793 - sparse_categorical_accuracy: 0.7297 - sparse_categorical_crossentropy: 0.8793 - val_loss: 0.7920 - val_sparse_categorical_accuracy: 0.7464 - val_sparse_categorical_crossentropy: 0.7920 - lr: 0.0010


In [10]:
model = create_bidirectional_model(2)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft_1 (TFOpLambda)  (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs_1 (TFOpLambda)     (None, 125, 513)     0           ['tf.signal.stft_1[0][0]']       
                                                                                                  
 tf.tensordot_1 (TFOpLambda)    (None, 125, 80)      0           ['tf.math.abs_1[0][0]']          
                                                                                            

In [11]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3,
                                        min_lr=0.00001, verbose=1)

In [12]:
del model
gc.collect()

14815

## Experiments

Training will be repeated 5 times with different weights initialization. For each number of layers, we created another directory with results.

In [13]:
TRAINING_SEEDS = list(range(5))
for i in range(1, 3):
    results = []
    EXPERIMENT_NAME = f"stacked_{i}_lstm"
    if os.path.exists(EXPERIMENT_NAME):
        shutil.rmtree(EXPERIMENT_NAME)
        os.mkdir(EXPERIMENT_NAME)
    else:
        os.mkdir(EXPERIMENT_NAME)

    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)

        model = create_bidirectional_model(i)

        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.01),
            loss=losses.SparseCategoricalCrossentropy(),
            metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
        )

        history = model.fit(
            train_ds,
            epochs=MAX_EPOCHS,
            validation_data=val_ds,
            shuffle=True,
            callbacks=[early_stopping, reduce_lr]
        )

        with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
            pickle.dump(history.history, file)

        eval_results = model.evaluate(test_ds)

        predictions = model.predict(test_ds)
        with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
            pickle.dump(predictions, file)

        results += [{
            'seed': seed,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        gc.collect()

    results_temp = pd.DataFrame(results)
    results_df = pd.concat([results_temp.drop(["results"], axis=1), results_temp["results"].apply(pd.Series)], axis=1)
    results_df.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 16s 139ms/step - loss: 1.3532 - sparse_categorical_accuracy: 0.6428 - sparse_categorical_crossentropy: 1.3532 - val_loss: 1.0181 - val_sparse_categorical_accuracy: 0.6886 - val_sparse_categorical_crossentropy: 1.0181 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 13s 137ms/step - loss: 0.8273 - sparse_categorical_accuracy: 0.7439 - sparse_categorical_crossentropy: 0.8273 - val_loss: 0.7619 - val_sparse_categorical_accuracy: 0.7589 - val_sparse_categorical_crossentropy: 0.7619 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 14s 138ms/step - loss: 0.6551 - sparse_categorical_accuracy: 0.7939 - sparse_categorical_crossentropy: 0.6551 - val_loss: 0.5667 - val_sparse_categorical_accuracy: 0.8207 - val_sparse_categorical_crossentropy: 0.5667 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 14s 143ms/step - loss: 0.5387 - sparse_categorical_accuracy: 0.8291 - sparse_categorical_crossentr

In [14]:
results_df